# first actual transformer

In [1]:
import tiktoken
import torch 
import datasets
import random

# r50k_base vocab size: 50,257 https://arxiv.org/pdf/2404.09894
enc = tiktoken.get_encoding("r50k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

def encode(string):
    return torch.tensor(enc.encode(string))

def decode(tensor):
    return enc.decode(tensor.cpu().squeeze().numpy())
    
dataset = datasets.load_dataset('karpathy/tiny_shakespeare')
dataset_tok = dataset.map(lambda row: {"tok": torch.tensor(encode(row["text"]), device="mps")}, remove_columns="text")

def get_sample(split, sample_length, batch_size):
    tokens = dataset_tok[split]["tok"][0]
    idcs = torch.randint(len(tokens)-sample_length, (batch_size,))
    x = torch.stack([torch.tensor(tokens[x:x+sample_length]) for x in idcs])
    y = torch.stack([torch.tensor(tokens[x+1:x+sample_length+1]) for x in idcs])
    return x, y

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})

In [5]:
get_sample("train", 10, 4)

(tensor([[  705, 48010,   257,  8046,  1165,  1165,  8593, 19917,   540,     0],
         [ 4161, 15816,   351,   269,  8516,    11,   198,  1722,   331,  8623],
         [  849, 22096, 48586,  2637,   198,   198,  5962,  2629,  6749,    25],
         [ 2767,    25,   198,  2990,   389,   379,  1021,    11,   290,   345]]),
 tensor([[48010,   257,  8046,  1165,  1165,  8593, 19917,   540,     0,   198],
         [15816,   351,   269,  8516,    11,   198,  1722,   331,  8623, 10846],
         [22096, 48586,  2637,   198,   198,  5962,  2629,  6749,    25,   198],
         [   25,   198,  2990,   389,   379,  1021,    11,   290,   345,  2236]]))

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        # out = F.scaled_dot_product_attention(q, k, v)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [3]:
vocab_size = 50_272
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
batch_size = 64
block_size = 128 
device = "mps"
learning_rate = 3e-4

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


49.34896 M parameters


In [7]:
import time

log_interval = 10
eval_interval = 50
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

curr_time = time.time()
tok_total = 0 

for curr_step in range(1, 1000):

    # every once in a while evaluate the loss on train and val sets
    #     losses = estimate_loss()
    #     print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_sample('train', block_size, batch_size)
    tok_step = xb.view(-1).size(0)
    tok_total += tok_step

    # evaluate the loss
    logits, loss = model(xb.to(device), yb.to(device))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    step_time = time.time() - curr_time
    if curr_step % log_interval == 0:
        print(f"step {curr_step}: loss {loss.detach().item():.2f} {step_time*1000:.2f}ms/step {(tok_step/step_time):.0f}tok/s (total {tok_total:,} tok)")

    if curr_step % eval_interval == 0:
        model.eval()
        xb, yb = get_sample('validation', block_size, batch_size)
        logits, loss = model(xb.to(device), yb.to(device))
        print(f"step {curr_step} eval: val_loss {loss.detach().item():.2f}")
        # with torch.no_grad():
        #     output = model.generate(encode("hello my name is").to(device).unsqueeze(0), 10)
        #     print(decode(output))
        model.train()
    curr_time = time.time()


step 10: loss 3.26 1218.93ms/step 6721tok/s (total 81,920 tok)
step 20: loss 3.19 1216.97ms/step 6731tok/s (total 163,840 tok)
step 30: loss 3.19 1220.66ms/step 6711tok/s (total 245,760 tok)
step 40: loss 3.14 1224.20ms/step 6692tok/s (total 327,680 tok)
step 50: loss 3.18 1216.49ms/step 6734tok/s (total 409,600 tok)
step 50 eval: val_loss 4.56
step 60: loss 3.06 1206.55ms/step 6790tok/s (total 491,520 tok)
step 70: loss 3.00 1212.93ms/step 6754tok/s (total 573,440 tok)
step 80: loss 3.12 1213.91ms/step 6748tok/s (total 655,360 tok)
step 90: loss 2.99 1225.76ms/step 6683tok/s (total 737,280 tok)
step 100: loss 3.02 1225.65ms/step 6684tok/s (total 819,200 tok)
step 100 eval: val_loss 4.47
step 110: loss 3.00 1240.91ms/step 6602tok/s (total 901,120 tok)
step 120: loss 3.03 1236.63ms/step 6624tok/s (total 983,040 tok)
step 130: loss 3.05 1276.46ms/step 6418tok/s (total 1,064,960 tok)
step 140: loss 2.93 1199.42ms/step 6830tok/s (total 1,146,880 tok)
step 150: loss 2.95 1204.45ms/step 6801

KeyboardInterrupt: 

In [9]:
with torch.no_grad():
    output = model.generate(encode("First Citizen:").to(device).unsqueeze(0), 100)
    print(decode(output))


First Citizen:
No, why thus the city's open
In him when his power we have caused his success.

Girl:
Come, man, standfully:
Take you with us, we two stock,
Ascreature in his brows with child,
And then receive alike together:
And so your fortune's ransom.

KING EDWARD IV:
At the other chase, 'tis the devil
The instruments of few and leave to breathe the ug perfect day


In [56]:
with torch.no_grad():
    output = model.generate(encode("First Citizen:").to(device).unsqueeze(0), 50)
    print(decode(output))


First Citizen:
Old replies: what comest beauty in,
Gardicked art rather have madeImmediately their patron: many
Commit takiss: I,--not we buckle up this case
Of hid, do add private well and go
